In [3]:
# Dependencies
import json
import requests
import pprint


In [4]:
startPeriod = str(2020)

In [5]:
#ABS config
dataflowIdentifier = "C21_G34_CED"
# C21_G34_CED = Census 2021, G34 Number of motor vehicles by dwellings, Commonwealth Electoral Divisions
# C21_G34_SA2 = Census 2021, G34 Number of motor vehicles by dwellings, Main Statistical Areas Level 2 and up 
# C21_G34_SUA = Census 2021, G34 Number of motor vehicles by dwellings, Significant Urban Areas 


dataKey = "all"
startPeriod = "2020"
endPeriod = "2023"
detail = "full"
dimensionAtObservation = "AllDimensions"


url = "https://api.data.abs.gov.au/data/"
query = url + dataflowIdentifier + "/" + dataKey + "?startPeriod=" + startPeriod + "&endPeriod=" + endPeriod + "&detail=" + detail + "&DimensionAtObservation=" + dimensionAtObservation + "&format=jsondata"







In [6]:
ABS_response = requests.get(query)
ABS_json = ABS_response.json()

print(json.dumps(ABS_json, indent=4))


{
    "meta": {
        "schema": "https://raw.githubusercontent.com/sdmx-twg/sdmx-json/master/data-message/tools/schemas/1.0/sdmx-json-data-schema.json",
        "id": "IREF017037",
        "prepared": "2023-01-23T23:33:05Z",
        "test": false,
        "contentLanguages": [
            "en"
        ],
        "sender": {
            "id": "_Stat_V8",
            "name": "unknown",
            "names": {
                "en": "unknown"
            }
        }
    },
    "data": {
        "dataSets": [
            {
                "action": "Information",
                "links": [
                    {
                        "urn": "urn:sdmx:org.sdmx.infomodel.datastructure.DataStructure=ABS:C21_G34_CED(1.0.0)",
                        "rel": "DataStructure"
                    }
                ],
                "annotations": [
                    0,
                    1,
                    2,
                    3,
                    4,
                    5,
             